<a href="https://colab.research.google.com/github/mzsmarcos/projeto_pipelineETL/blob/main/Etapa_6_jun%C3%A7%C3%A3o_parte_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Junção parte II**

Nesse notebook estão os códigos da segunda parte da junção dos arquivos

-Enriquecimento dos dados

Dataframe final : df_empresas_brasil

1 - df_empresas_brasil + df_natureza

2 - df_empresas_brasil + df_municipio

3 - df_empresas_brasil + df_motivos

4 - df_empresas_brasil + df_pais

5 - df_empresas_brasil + df_cnae

-Geração do dataframe final

Configuração do PySpark -  instalar o PySpark no Colab

In [ ]:
# Instalar o PySpark
!pip install pyspark


Executar a instalação do PySpark e configurar o ambiente Spark.

Isso é feito definindo algumas variáveis de ambiente para garantir que o Spark funcione corretamente no Colab.

In [ ]:
# Instalar Java 8
!apt-get install openjdk-8-jdk -y

# Baixar o Apache Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# Extrair o Apache Spark
!tar xf spark-3.5.0-bin-hadoop3.tgz

# Definir variáveis de ambiente
import os # Módulo para interagir com o sistema operacional, como manipulação de arquivos e diretórios
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

# Instalar as bibliotecas PySpark e Findspark
!pip install -q pyspark==3.5.0 findspark



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-8-jdk is already the newest version (8u442-b06~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
# Verificar se o arquivo foi baixado
!ls -lh


total 764M
drwx------  6 root root 4.0K Mar 17 23:17 drive
drwxr-xr-x  1 root root 4.0K Mar 14 13:32 sample_data
drwxr-xr-x 13 1000 1000 4.0K Sep  9  2023 spark-3.5.0-bin-hadoop3
-rw-r--r--  1 root root 382M Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz
-rw-r--r--  1 root root 382M Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz.1


Criar a Sessão Spark

In [ ]:
# Importar as bibliotecas
import findspark # Permite localizar e configurar o Spark no ambiente, garantindo que o PySpark possa ser usado
findspark.init("/content/spark-3.5.0-bin-hadoop3")

from pyspark.sql import SparkSession  # Importa a classe SparkSession para criar uma sessão do Spark e interagir com DataFrames

# Criar a sessão Spark
spark = SparkSession.builder \
    .appName("PySparkExample") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Verificar se a sessão foi criada corretamente
print("Versão do Spark:", spark.version)


Versão do Spark: 3.5.0


In [ ]:
# Os
import os

In [ ]:
# Montando o Google Drive no Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, col, when, broadcast
from pyspark.sql.types import ArrayType, StringType

# Configurações de memória do Spark
spark = SparkSession.builder \
    .appName("Projeto ETL") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Carrega os DataFrames necessários
df_empresas_brasil_1 = spark.read.parquet("/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/df_empresas_brasil_1")
df_motivos = spark.read.parquet("/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/motivos.parquet")
df_paises = spark.read.parquet("/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/paises.parquet")
df_cnae = spark.read.parquet("/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/cnae.parquet")
df_municipios = spark.read.parquet("/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/municipios.parquet")
df_natureza = spark.read.parquet("/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/natureza.parquet")
df_qualificacao = spark.read.parquet("/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/qualificacao.parquet")

# 1. Concatenar as colunas referentes ao CNPJ
df_empresas_brasil = df_empresas_brasil_1.withColumn("cnpj", concat_ws("", col("cnpj_basico"), col("cnpj_ordem"), col("cnpj_dv")))

# 2. Identificar a situação cadastral
df_empresas_brasil = df_empresas_brasil.withColumn("sit_cadastral",
    when(col("cod_sit_cadastral") == "01", "NULA")
    .when(col("cod_sit_cadastral") == "02", "ATIVA")
    .when(col("cod_sit_cadastral") == "03", "SUSPENSA")
    .when(col("cod_sit_cadastral") == "04", "INAPTA")
    .when(col("cod_sit_cadastral") == "08", "BAIXADA")
    .otherwise("DESCONHECIDA")
)

# 3. Identificar o motivo da situação cadastral (com tratamento de nulos)
df_empresas_brasil = (
    df_empresas_brasil
    .join(broadcast(df_motivos), df_empresas_brasil["cod_mot_sit_cadastral"] == df_motivos["cod_motivo"], "left")
    .drop("cod_mot_sit_cadastral")
    .withColumnRenamed("descricao_motivo", "motivo_sit_cadastral")
    .na.fill({"motivo_sit_cadastral": "SEM MOTIVO"})
)

# 4. Identificar o país (com tratamento de nulos e broadcast join)
df_empresas_brasil = (
    df_empresas_brasil
    .join(broadcast(df_paises), df_empresas_brasil["cod_pais"] == df_paises["cod_pais"], "left")
    .drop("cod_pais")
    .withColumnRenamed("nome_pais", "pais")
    .na.fill({"pais": "NÃO APLICADO"})
)

# 5. Identificar a CNAE principal (com tratamento de nulos e broadcast join)
df_empresas_brasil = (
    df_empresas_brasil
    .join(broadcast(df_cnae), df_empresas_brasil["cnae_fiscal_principal"] == df_cnae["cod_cnae"], "left")
    .drop("cnae_fiscal_principal")
    .withColumnRenamed("descricao", "cnae_principal")
    .na.fill({"cnae_principal": "CNAE PRINCIPAL DESCONHECIDO"})
)

# 6. Identificar o município (com tratamento de nulos e broadcast join)
df_empresas_brasil = (
    df_empresas_brasil
    .join(broadcast(df_municipios), df_empresas_brasil["municipio"] == df_municipios["cod_municipio"], "left")
    .drop("municipio")
    .withColumnRenamed("nome_municipio", "municipio")
    .na.fill({"municipio": "MUNICÍPIO DESCONHECIDO"})
)

# 7. Identificar a natureza jurídica (com tratamento de nulos e broadcast join)
df_empresas_brasil = (
    df_empresas_brasil
    .join(broadcast(df_natureza), df_empresas_brasil["cod_nat_juridica"] == df_natureza["cod_nat_juridica"], "left")
    .drop("cod_nat_juridica")
    .withColumnRenamed("nome_nat_juridica", "nat_juridica")
    .na.fill({"nat_juridica": "NAT JURÍDICA DESC"})
)

# 8. Identificar a qualificação do responsável (com tratamento de nulos e broadcast join)
df_empresas_brasil = (
    df_empresas_brasil
    .join(broadcast(df_qualificacao), df_empresas_brasil["cod_qualificacao_responsavel"] == df_qualificacao["cod_quali_socio"], "left")
    .drop("cod_qualificacao_responsavel")
    .withColumnRenamed("nome_quali_socio", "qualificacao_responsavel")
    .na.fill({"qualificacao_responsavel": "DESCONHECIDA"})
)

# Salvamento do DataFrame final
output_path = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/df_empresas_brasil"

# Reparticionamento e salvamento
df_empresas_brasil.repartition(200) \
    .write \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .parquet(output_path)

print(f"DataFrame final salvo com sucesso em: {output_path}")

DataFrame final salvo com sucesso em: /content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/df_empresas_brasil


In [ ]:
df_empresas_brasil.show(n=20, truncate=False)

+-----------+----------+-------+----------------+--------------------------------------------+-----------------+------------------+---------------+---------------------+-----------------------------------------------+--------------+----------------------------------------+------+------------------------------------------------------------------------------+----------------------------+--------+---+------------+------------+------------+------------+------------+------------+-----------------------------------------+-----------------+----------------------+------------------+--------------------------------------------------------------------------------------+--------------------------+---------------------+----------------------------+--------------------+-----------------------------+------------------------+-----------------------------+----------------------+------------------------------------------+---------------------------------------------+--------------+-----------------+---

In [ ]:
# Obtém o número de linhas
df_empresas_brasil.take(1) # Força a excecução antes do count
num_linhas = df_empresas_brasil.count()

# Obtém o número de colunas
num_colunas = len(df_empresas_brasil.columns)

print(f"Número de linhas: {num_linhas}")
print(f"Número de colunas: {num_colunas}")

Número de linhas: 64017370
Número de colunas: 59
